In [0]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from fastai.vision import *
from fastai.vision.gan import *
from urllib.parse import urlparse
import torch.nn as nn
import torch.nn.functional as F

In [0]:
# PLATFORM DEPENDENT CODE !!!!!
base_dir = '/content'

if (importlib.util.find_spec("google.colab") is not None):
  from google.colab import drive
  drive.mount('/content/gdrive', force_remount=True)
  root_dir = "/content/gdrive/My Drive/"
  base_dir = root_dir + 'ML/'

Mounted at /content/gdrive


Load a pretrained discriminator. Pending research about whether it's good to start with a pretrained one, kind of more knolewdgeable than the generator.

In [0]:
critic = load_learner(base_dir, 'faceBinaryClassifierResnet18_1.pkl')

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:454: SourceChangeWarning: source code of class 'fastai.layers.AdaptiveConcatPool2d' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:454: SourceChangeWarning: source code of class 'fastai.layers.Flatten' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)


In [0]:
def loadNewTestImage(url: str, filename: str, testRootDir:Path):
  parsedUrl = urlparse(url)
  ext = os.path.splitext(parsedUrl.path)[1]
  path = testRootDir/(filename + ext)
  download_url(url, path)
  img = open_image(path)
  return img

In [0]:
img = loadNewTestImage('https://live.staticflickr.com/3844/14761664536_f875cf6b9c_b.jpg', 'fakeFace', Path('/content'))
img

In [0]:
critic.predict(img)

(Category negative, tensor(0), tensor([0.9978, 0.0022]))

In [0]:
#Testing a transpose convolution
input = torch.rand(64, 100, 1, 1)
nn.ConvTranspose2d(100, 1024, 4, padding=0, stride=1)(input).size()

torch.Size([64, 1024, 4, 4])

In [0]:
class ConvBlock(nn.Module):
  def __init__(self, in_channels, out_channels):
    super(ConvBlock, self).__init__()
    self.conv = nn.Sequential(
      nn.ConvTranspose2d(in_channels, out_channels, (4, 4), padding=1, stride=2),
      nn.BatchNorm2d(out_channels),
      nn.ReLU()
    )
    
  def forward(self, x):
    return self.conv(x)

class GeneratorNet(nn.Module):
  def __init__(self):
    super(GeneratorNet, self).__init__()
    self.layers = nn.Sequential(
      #nn.Linear(100, 1024 * 4 * 4),
      nn.ConvTranspose2d(100, 1024, 4, padding=0, stride=1),
      ConvBlock(1024, 512),
      ConvBlock(512, 256),
      ConvBlock(256, 128),
      #ConvBlock(128, 64)
      nn.ConvTranspose2d(128, 3, 4, padding=1, stride=2),
      nn.Tanh()
    )

  def forward(self, x):
    # x = x.view(1024, 4, 4) Use it when putting FC at the beginning
    x = self.layers(x)
    return x

In [12]:
#Testing a forward pass through the generator
input = torch.rand(64, 100, 1, 1)
input = GeneratorNet().forward(input)
input.size()

torch.Size([64, 3, 64, 64])

In [0]:
??GANLearner

In [0]:
learner = GANLearner()

PENDING:

*  What if I make the final convolution a normal convolution with in_channels=out_channels (probably together with uncommenting the last ConvBlock call)? 
*   Try replacing first convolution with FC layer + reshape (view)
*  Further research about that first layer "project and reshape"
*  Using a pretrained critic might be a bad idea, especially thinking about the gradients used by the generator to improve